EECS 4412 Project - Phase II
Maryam Salarian
Analysis of the dataset + preprocessing + ....

In [1]:
# 1. load the dataset

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
reviews = pd.read_csv("/content/drive/MyDrive/EECS4412/data/project/Books_rating.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 2. display the dataset to get an idea

reviews.head(10)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
5,0826414346,Dr. Seuss: American Icon,NaN,A2F6NONFUDB6UK,Malvin,2/2,4.0,1127174400,One of America's greatest creative talents,"""Dr. Seuss: American Icon"" by Philip Nel is a ..."
6,0826414346,Dr. Seuss: American Icon,NaN,A14OJS0VWMOSWO,Midwest Book Review,3/4,5.0,1100131200,A memorably excellent survey of Dr. Seuss' man...,Theodor Seuss Giesel was best known as 'Dr. Se...
7,0826414346,Dr. Seuss: American Icon,NaN,A2RSSXTDZDUSH4,J. Squire,0/0,5.0,1231200000,Academia At It's Best,When I recieved this book as a gift for Christ...
8,0826414346,Dr. Seuss: American Icon,NaN,A25MD5I2GUIW6W,"J. P. HIGBED ""big fellow""",0/0,5.0,1209859200,And to think that I read it on the tram!,Trams (or any public transport) are not usuall...
9,0826414346,Dr. Seuss: American Icon,NaN,A3VA4XFS5WNJO3,Donald Burnside,3/5,4.0,1076371200,Fascinating account of a genius at work,"As far as I am aware, this is the first book-l..."


In [3]:
# 3. as displayed, some of the books have multiple reviews and ratings
# handling approach: treat each review as a separate sample, and once sentiment assigned, take majority vote
# alternative to majority vote: take weighted sentiment, using review/helpfulness as weight, and assigning small weight to those with review/helpfulness = 0.
# size of the unaltered dataset
print("size of unaltered reviews dataset:", reviews.shape)

# checking data types, as non-numeric data type for rating will affect preprocessing
print ("\ndata type of attributes:\n", reviews.dtypes)

# rename relevant attrs for ease of reference: Title to title, review/score to rating, review/helpfulness to helpfulness, review/text to review
reviews.rename(columns={'Title':'title', 'review/score': 'rating', 'review/helpfulness': 'helpfulness', 'review/text':'review'}, inplace=True)

# drop all other cols
columns_to_keep = ["title", "rating", "helpfulness", "review"]
reviews = reviews[columns_to_keep]
print("size of dataset after dropping 6 out of 10 attributes:", reviews.shape)

# check the max and min value for rating (review/score) in the dataset
print("\nmin rating:", reviews["rating"].min())
print("max rating:", reviews["rating"].max())


size of unaltered reviews dataset: (3000000, 10)

data type of attributes:
 Id                     object
Title                  object
Price                 float64
User_id                object
profileName            object
review/helpfulness     object
review/score          float64
review/time             int64
review/summary         object
review/text            object
dtype: object
size of dataset after dropping 6 out of 10 attributes: (3000000, 4)

min rating: 1.0
max rating: 5.0


In [6]:
# 4. add new col, sentiment, based on review rating
# will be added to the entire dataset as target attribute
# rating >= 4 is positive: +1
# rating =< 2 is negative: -1
# rating =3 is netural: 0

# perform rating-specific preprocessing:

# a) convert rating to numreic -> from above data types we conclude that rating is already in numeric format: float64
# b) remove rows with missing or NaN rating
reviews = reviews.dropna(subset=["rating"])
# c) ensure all ratings are within range -> from above rating range, we confirm that all ratings are within [1,5]

# generate the new sentiment column
def assign_sentiment(x):
    if x >= 4:
        return 1
    elif x == 3:
        return 0
    else:
        return -1

# apply the above function to every value in the rating col, and store result in new col
reviews["sentiment"] = reviews["rating"].apply(assign_sentiment)

# convert helpfulness from object to numeric values, fill NaN with 0
def fraction_to_float(x):
    if isinstance(x, str) and "/" in x:
        try:
            num, denom = x.split("/")
            return float(num) / float(denom)
        except:
            return np.nan  # invalid fraction
    else:
        # try to convert directly to float
        return pd.to_numeric(x, errors='coerce')
reviews["helpfulness"] = reviews["helpfulness"].apply(fraction_to_float)
reviews["helpfulness"] = reviews["helpfulness"].fillna(0)

print("size of dataset after removing NaN ratings and adding new col:", reviews.shape)
reviews.head(5)


size of dataset after removing NaN ratings and adding new col: (2999992, 5)


,title,rating,helpfulness,review,sentiment
0,Its Only Art If Its Well Hung!,4.0,1.000000,This is only for Julie Strain fans. It's a col...,1
1,Dr. Seuss: American Icon,5.0,1.000000,I don't care much for Dr. Seuss but after read...,1
2,Dr. Seuss: American Icon,5.0,0.909091,"If people become the books they read and if ""t...",1
3,Dr. Seuss: American Icon,4.0,1.000000,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",1
4,Dr. Seuss: American Icon,4.0,1.000000,Philip Nel - Dr. Seuss: American IconThis is b...,1


In [7]:
# 5. perform further preprocessing on the dataset

# a) remove rows with missig reviews: NaN, '', ""
# drop rows where review is NaN
reviews = reviews.dropna(subset=["review"])
# remove empty strings or whitespace-only reviews
df = reviews[reviews["review"].str.strip().astype(bool)]
print("size of dataset after removing NaN or empty reviews:", reviews.shape)

# b) remove identical rows with duplicate review per book
reviews = reviews.drop_duplicates(subset=["title", "review"])
print("size of dataset after dropping duplicates:", reviews.shape)

# c) remove leading and trailing white spaces + normalize space between chars
reviews["review"] = reviews["review"].str.replace(r"\s+", " ", regex=True).str.strip()

# d) to prevent overfitting and reduce noise, remove reviews with < 2 words
reviews = reviews[reviews["review"].str.len() >= 2]

# d) lower case all characters in review
reviews["review"] = reviews["review"].str.lower()

# e) remove non-text/num chars, keep the spaces
reviews["review"] = reviews["review"].str.replace(r"[^a-zA-Z0-9\s']", " ", regex=True)

# trash bin:
# reviews = reviews.drop_duplicates()


size of dataset after removing NaN or empty reviews: (2999992, 5)
size of dataset after dropping duplicates: (2616740, 5)


In [ ]:
# 6. perfrom text - order them appropriately
# stop word removal
# expand contrations
# tokenization
# lemmatization
# remove rare words - unsupervised feature selection DF
# supervised feature selection using chi square
# validate feature selection on cross-validation folds

# then move to the model
